In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import datetime as dt
import numpy as np

In [ ]:
now = dt.datetime.now()
now = now.strftime('%Y-%m-%d')

In [ ]:
list_path = f'/finngen/green/austina/olink_names_oct_30_2023.csv'
olink_names = pd.read_csv(list_path, header=None)
olink_names = list(olink_names[0])

remove_prots = [
    # proteins in UKB but not CKB
    'HLA_A',
    'ERVV_1',
    
    # proteins in CKB but not UKB
    'CD97',
    'FGFR1OP',
    'LRMP',
    'CASC4',
    'DARS',
    'HARS',
    'WISP2',
    'FOPNL',
    'WISP1',
    
    # proteins not in FinnGen
    'EDEM2',
    'EP300',
    'CGA',
    'CDHR1',
    'CPLX2',
    'CLSTN1',
    'IFIT1',
    'FGF3',
    'TAGLN3',
    'YAP1',
    'ADIPOQ',
    'BCL2L11',
    'BMP6',
    'BID',
    'SH3GL3',
    'ARL13B',
    'ANGPTL7',
    'MGLL',
    'MPI',
    'MAGEA3',
    'KCNH2',
    
    # proteins missing > 20%
    'GLIPR1', 
    'NPM1', 
    'PCOLCE' 
]


olink_names = [prot for prot in olink_names if prot not in remove_prots]

# Load pred dfs

In [ ]:
## lightgbm boruta
name = '/finngen/green/austina/pAge_UKB_3k_post_Boruta_scores_UKB_dart_2023-12-28.csv'
# name = f'{filepath}results/lgbm/pAge_UKB_3k_dart_noImp_minMax_scores_UKB_2024-01-05.csv'
pred_df_ukb = pd.read_csv(name)

## lightgbm in top 20 proteins
name = '/finngen/green/austina/pAge_UKB_3k_top_20_scores_UKB_2023-12-29.csv'
pred_df_ukb_20 = pd.read_csv(name)

# merge
pred_df_ukb_20 = pred_df_ukb_20.rename(columns={'predicted_values':'predicted_values_20'})
pred_df_ukb_20 = pred_df_ukb_20[['eid', 'predicted_values_20']]
pred_df_ukb = pd.merge(pred_df_ukb, pred_df_ukb_20, on='eid', how='inner')

# lightgbm boruta in CKB
name = '/finngen/green/austina/pAge_CKB_3k_scores_all_dart_2023-12-28.csv'
# name = '/Users/aargenti/Documents/proteomic_age/results/lgbm/pAge_UKB_3k_dart_noImp_minMax_scores_CKB_all_2024-01-05.csv'
pred_df_ckb = pd.read_csv(name)

# Load UKB data

In [ ]:
# import data
data_path2 = '/finngen/green/austina/olink_data_imputed_nov_18_2023.csv'
ukb_olink_data = pd.read_csv(data_path2)
# survival_data = survival_data.drop('recruitment_age', axis=1)

exposure_path = '/finngen/green/austina/ukb_imputation1_jul_25_2023.feather'
exposure_data = pd.read_feather(exposure_path)
exposure_data.rename(columns={'SHBG': 'SHBG_biochem'}, inplace=True)

# dx_path = f'{filepath}data/interview_dx_data.csv'
# dx_data = pd.read_csv(dx_path)

disease_path = '/finngen/green/austina/incident_disease_data_june_21_2023.csv'
disease_data = pd.read_csv(disease_path)
disease_cols = list(disease_data.columns)

mort_path = '/finngen/green/austina/mortality_data_oct_18_2023.csv'
mort_data = pd.read_csv(mort_path)

cancer_path = '/finngen/green/austina/cancer_data_july_05_2023.csv'
cancer_data = pd.read_csv(cancer_path)

survival_data = pd.merge(pred_df_ukb, ukb_olink_data, on='eid', how='inner')
survival_data = pd.merge(survival_data, exposure_data, on='eid', how='left')
# survival_data = pd.merge(survival_data, dx_data, on='eid', how='inner')
survival_data = pd.merge(survival_data, mort_data, on='eid', how='inner')
survival_data = pd.merge(survival_data, disease_data, on='eid', how='inner')
survival_data = pd.merge(survival_data, cancer_data, on='eid', how='inner')

# convert survival time in days to years
survival_data['ACM_survival_years'] = survival_data['ACM_survival_time'] / 365.25

del cancer_data
# del dx_data
del disease_data

len(survival_data.index)


# Load CKB data

In [ ]:
# import data

# path to data
data_path = '/finngen/green/austina/ckb_coded_olink_oct_17_2023.csv'
# load data
ckb_data = pd.read_csv(data_path)
# subset to those in random subset
# ckb_data = ckb_data[ckb_data['olinkexp1536_chd_b1_subcohort'] == 1]
ckb_data = ckb_data.drop('is_female', axis=1)

# path to data
data_path = '/finngen/green/austina/ckb_coded_data_oct_24_2023.feather'
# data_path = '/finngen/green/austina/ckb_coded_data_oct_24_2023.csv'
# load data
ckb_mort_data = pd.read_feather(data_path)
# ckb_mort_data = pd.read_csv(data_path)

ckb_data = pd.merge(ckb_data, ckb_mort_data, on='csid', how='inner')
ckb_data = pd.merge(ckb_data, pred_df_ckb, on='csid', how='left')

len(ckb_data.index)

del ckb_mort_data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# the scaler object (model)
scaler = MinMaxScaler()

# copy data
ckb_normalized = ckb_data.copy()

# UKB
for protein in olink_names:
    # fit and transform the data
    value = scaler.fit_transform(ckb_normalized[protein].values.reshape(-1, 1)) 
    ckb_normalized[protein] = value
    
    # Calculate the median
    median = ckb_normalized[protein].median()
    # Median center
    ckb_normalized[protein] = ckb_normalized[protein] - median

# Load FinnGen data

In [ ]:
file_path = '/home/ivm/Documents/olink_imputed_jan_25_2024.csv'
finngen_olink = pd.read_csv(file_path)
file_path = '/finngen/library-red/EA5/proteomics/olink/third_batch/QCd/covars.txt'
covars2 = pd.read_csv(file_path, sep='\t')

finngen_all = pd.merge(finngen_olink, covars2[['FID', 'FU_END_AGE', 'SEX']],  on='FID', how='inner')
finngen_all.set_index('FID', inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# the scaler object (model)
scaler = MinMaxScaler()

# copy data
olink_normalized = finngen_all.copy()

# UKB
for protein in olink_names:
    # fit and transform the data
    value = scaler.fit_transform(olink_normalized[protein].values.reshape(-1, 1)) 
    olink_normalized[protein] = value
    
    # Calculate the median
    median = olink_normalized[protein].median()
    # Median center
    olink_normalized[protein] = olink_normalized[protein] - median

In [ ]:
finngen_all['SEX'].value_counts()

# ProtAgeGap

In [ ]:
survival_data['ProtAgeAccel'] = survival_data['predicted_values'] - survival_data['age_granular']
ckb_normalized['ProtAgeAccel'] = ckb_normalized['predicted_values'] - ckb_normalized['age_granular']
survival_data['ProtAgeAccel_20'] = survival_data['predicted_values_20'] - survival_data['age_granular']

In [ ]:
ckb_normalized['ProtAgeAccel'].describe()

# Disease lists

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Define the number of colors
n = 3

# Get the Viridis colormap
cmap = cm.get_cmap('viridis')

# Get the desired colors from Viridis
colors = cmap(np.linspace(0, 1, n))

# Define the disease lists
ds_list = [
    'incident_diabetes',
    'incident_IHD',
    'incident_all_stroke',
    'incident_ischemic_stroke',
    'incident_COPD',
    'incident_liver',
    'incident_kidney',
    'incident_all_dementia',
    # incident_vasc_dementia,
    'incident_alzheimers',
    'incident_parkinsons',
    'incident_rheumatoid',
    'incident_macular',
    'incident_osteoporosis',
    'incident_osteoarthritis'
]

prev_list = [
    'prevalent_diabetes',
    'prevalent_IHD',
    'prevalent_all_stroke',
    'prevalent_ischemic_stroke',
    'prevalent_COPD',
    'prevalent_liver',
    'prevalent_kidney',
    'prevalent_all_dementia',
    # prevalent_vasc_dementia,
    'prevalent_alzheimers',
    'prevalent_parkinsons',
    'prevalent_rheumatoid',
    'prevalent_macular',
    'prevalent_osteoporosis',
    'prevalent_osteoarthritis'
]

disease_list2 = [
    "Type II diabetes",
    "Ischemic heart disease",
    "All stroke",
    "Ischemic stroke",
    "Emphysema, COPD",
    "Chronic liver diseases",
    "Chronic kidney diseases",
    "All-cause dementia",
    # "Vascular dementia",
    "Alzheimer's disease",
    "Parkinson's disease",
    "Rheumatoid arthritis",
    "Macular degeneration",
    "Osteoporosis",
    "Osteoarthritis"
]

# Get the counts of prevalent and incident cases for each disease
prev_counts = [survival_data[prev].sum() for prev in prev_list]
ds_counts = [survival_data[ds].sum() for ds in ds_list]

# Combine the disease names and counts into a list of tuples
data = list(zip(disease_list2, prev_counts, ds_counts))

# Sort the data based on the sum of prevalent and incident cases
sorted_data = sorted(data, key=lambda x: sum(x[1:]), reverse=True)

# Unpack the sorted data into separate lists
sorted_diseases, sorted_prev_counts, sorted_ds_counts = zip(*sorted_data)

# Load models

In [ ]:
import lightgbm as lgb
import pickle
   
with open("/finngen/green/austina/pAge_UKB_3k_dart_minMax_pre_boruta_2023-12-23.p", "rb") as f:
   lgbm_model = pickle.load(f)
   
# load data dictionary used to train lgbm model on server
with open('/finngen/green/austina/UKB_data_dict_dart_2023-12-23.p', "rb") as f:
   server_data = pickle.load(f)

# Figure 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib.ticker import FuncFormatter

sns.set_style("ticks")

# Set global font size
plt.rcParams.update({'font.size': 7})

# set inner boxplot params
inner_kws=dict(box_width=3, whis_width=1)

# Create a new figure and specify the layout using gridspec
# fig = plt.figure(figsize=(11, 10))
cm = 1/2.54 #centimeters in inches
fig = plt.figure(figsize=(19.8*cm, 18.0*cm))
gs = gridspec.GridSpec(3, 3, figure=fig, width_ratios=[1, 1, 1], height_ratios=[2, 2, 2])



# plot a: age plot in both
ax = plt.subplot(gs[0, 0], aspect='auto')  # Span the entire left column
ax.set_title('a', fontweight='bold', loc='left', fontsize=12)

# Plotting density curves for dataset1 and dataset2 with different fill colors
sns.kdeplot(pred_df_ukb['age_granular'], label='UKB', color='blue', fill=True, alpha=0.5, edgecolor='black')
sns.kdeplot(pred_df_ckb['age_granular'], label='CKB', color='red', fill=True, alpha=0.5, edgecolor='black')
sns.kdeplot(finngen_all['FU_END_AGE'], label='FinnGen', color='yellow', fill=True, alpha=0.5, edgecolor='black')

# Adding labels and title
plt.xlabel('Age at recruitment (years)')
plt.ylabel('Density')
plt.title('')

# Displaying the legend
plt.legend(frameon=False, bbox_to_anchor=(0.49,1))

# plot a: mortality age in both
ax = plt.subplot(gs[1, 0], aspect='auto')  # Span the entire left column
ax.set_title('b', fontweight='bold', loc='left', fontsize=12)

data1 = survival_data[survival_data['ACM_event_indicator'] == 1]
data2 = ckb_data[ckb_data['ACM_event_indicator'] == 1]

data1['censor_age'] = data1['age_granular'] + (data1['ACM_survival_time'] / 365.25)


# Plotting density curves for dataset1 and dataset2 with different fill colors
sns.kdeplot(data1['censor_age'], label='UKB', color='blue', fill=True, alpha=0.5, edgecolor='black')
sns.kdeplot(data2['censor_age'], label='CKB', color='red', fill=True, alpha=0.5, edgecolor='black')

# Adding labels and title
plt.xlabel('Age at death (years)')
plt.ylabel('Density')
plt.title('')

# Displaying the legend
plt.legend(frameon=False, bbox_to_anchor=(0.37,1))


# plot c: disease counts in UKB
ax = plt.subplot(gs[2, 0], aspect='auto')
ax.set_title('c', fontweight='bold', loc='left', fontsize=12)

# Set the y-axis positions
y_pos = np.arange(len(sorted_diseases))

# Plot the stacked horizontal bars
ax.barh(y_pos, sorted_prev_counts, align='center', color=colors[0], label='Prevalent', edgecolor='none')
ax.barh(y_pos, sorted_ds_counts, align='center', color=colors[1], label='Incident', left=sorted_prev_counts, edgecolor='none')

# Set the y-axis tick positions and labels
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_diseases)

# add thousands separator to x axis labels
ax.get_xaxis().set_major_formatter(
    FuncFormatter(lambda x, p: format(int(x), ',')))


# Set the x-axis label
ax.set_xlabel('UK Biobank disease cases')

# Set the title and legend
ax.legend(frameon=False)


# plot d: R2 plot in UKB
ax = plt.subplot(gs[0, 1], aspect='auto')
ax.set_title('d', fontweight='bold', loc='left', fontsize=12)

# get predictions
predictions_lgbm = lgbm_model.predict(server_data['X_test'])

# Evaluation metrics
r, pvalue = pearsonr(server_data['y_test'], predictions_lgbm)
r2 = r2_score(server_data['y_test'], predictions_lgbm)
rmse = mean_squared_error(server_data['y_test'], predictions_lgbm, squared=False)
mae = mean_absolute_error(server_data['y_test'], predictions_lgbm)

# plot predicted age against age
regplot = sns.regplot(
    x=server_data['y_test'], 
    y=predictions_lgbm,
    scatter_kws=dict(color='midnightblue', s=10, alpha=0.8),
    line_kws=dict(color='red')
)

# Set the title for the regplot
# ax.set_title('LightGBM - UKB')

# add annotation
annotation_text = f'r = {r:.2f}\nR² = {r2:.2f}\nRMSE = {rmse:.2f}\nMAE = {mae:.2f}'
plt.text(.05, .95, annotation_text, ha='left', va='top', transform=regplot.transAxes)
plt.text(.95, .1, 'UKB', ha='right', va='top', transform=regplot.transAxes)

# p-value = {pvalue:.2e} 
regplot.set(xlabel='Age at recruitment (years)', ylabel='ProtAge')


# plot e: R2 plot in CKB
ax = plt.subplot(gs[1, 1], aspect='auto')  # Span the entire left column
ax.set_title('e', fontweight='bold', loc='left', fontsize=12)

# get predictions
predictions_lgbm = lgbm_model.predict(ckb_normalized[olink_names])

# Evaluation metrics
r, pvalue = pearsonr(ckb_normalized['age_granular'], predictions_lgbm)
r2 = r2_score(ckb_normalized['age_granular'], predictions_lgbm)
rmse = mean_squared_error(ckb_normalized['age_granular'], predictions_lgbm, squared=False)
mae = mean_absolute_error(ckb_normalized['age_granular'], predictions_lgbm)

# plot predicted age against age
regplot = sns.regplot(
    x=ckb_normalized['age_granular'], 
    y=predictions_lgbm,
    scatter_kws=dict(color='darkred', s=10, alpha=0.8),
    line_kws=dict(color='red')
)

# Set the title for the regplot
# ax.set_title('LightGBM - UKB')

# add annotation
annotation_text = f'r = {r:.2f}\nR² = {r2:.2f}\nRMSE = {rmse:.2f}\nMAE = {mae:.2f}'
plt.text(.05, .95, annotation_text, ha='left', va='top', transform=regplot.transAxes)
plt.text(.95, .1, 'CKB', ha='right', va='top', transform=regplot.transAxes)

# p-value = {pvalue:.2e} 
regplot.set(xlabel='Age at recruitment (years)', ylabel='ProtAge')



# plot f: R2 plot in all
ax = plt.subplot(gs[2, 1], aspect='auto')  # Span the entire left column
ax.set_title('f', fontweight='bold', loc='left', fontsize=12)

# get predictions
predictions_lgbm = lgbm_model.predict(olink_normalized[olink_names])

# Evaluation metrics
r, pvalue = pearsonr(olink_normalized['FU_END_AGE'], predictions_lgbm)
r2 = r2_score(olink_normalized['FU_END_AGE'], predictions_lgbm)
rmse = mean_squared_error(olink_normalized['FU_END_AGE'], predictions_lgbm, squared=False)
mae = mean_absolute_error(olink_normalized['FU_END_AGE'], predictions_lgbm)

# plot predicted age against age
regplot = sns.regplot(
    x=olink_normalized['FU_END_AGE'], 
    y=predictions_lgbm,
    scatter_kws=dict(color='orange', s=10, alpha=0.8),
    line_kws=dict(color='red')
)

# Set the title for the regplot
# ax.set_title('LightGBM - UKB')

# add annotation
annotation_text = f'r = {r:.2f}\nR² = {r2:.2f}\nRMSE = {rmse:.2f}\nMAE = {mae:.2f}'
plt.text(.05, .95, annotation_text, ha='left', va='top', transform=regplot.transAxes)
plt.text(.95, .1, 'FinnGen', ha='right', va='top', transform=regplot.transAxes)

regplot.set(xlabel='Age at recruitment (years)', ylabel='ProtAge')

# plot e: ProtAgeAccel distribution  in both
ax = plt.subplot(gs[0, 2], aspect='auto')  # Span the entire left column
ax.set_title('g', fontweight='bold', loc='left', fontsize=12)

ckb_normalized['sex'] = ckb_normalized['is_female']

# Recode female as 1 and male as 0
recode_mapping = {'Yes': 'Female', 'No': 'Male'}
ckb_normalized['sex'] = ckb_normalized['sex'].replace(recode_mapping)

recode_mapping = {'female': 'Female', 'male': 'Male'}
olink_normalized['sex'] = olink_normalized['SEX'].replace(recode_mapping)
olink_normalized['ProtAge'] = predictions_lgbm
olink_normalized['ProtAgeAccel'] = olink_normalized['ProtAge'] - olink_normalized['FU_END_AGE']

# Combine the datasets into a single DataFrame
combined_data = pd.concat([survival_data.assign(dataset='UKB'), ckb_normalized.assign(dataset='CKB'), olink_normalized.assign(dataset='FinnGen')])

# Create the violin plot with the 'dataset' column as the hue
sns.violinplot(
    data=combined_data, 
    y='ProtAgeAccel', 
    x='sex', 
    hue='dataset',
    dodge=True,
    palette='Blues',
    inner_kws=inner_kws,
    linewidth=1
)

# Set y-axis limits
plt.ylim(-42, 42)

# Set labels and title
plt.xlabel('')
plt.ylabel('ProtAgeGap (years)')
# Displaying the legend
# plt.legend(frameon=False, bbox_to_anchor=(1.02,0.32))
plt.legend(
    frameon=False, 
    bbox_to_anchor=(1.02,0.15), 
    ncols=3, 
    columnspacing=0.6, 
    handletextpad=0.2
)


# plot h: ProtAgeAccel distribution  in both
ax = plt.subplot(gs[1, 2], aspect='auto')  # Span the entire left column
ax.set_title('h', fontweight='bold', loc='left', fontsize=12)

sns.violinplot(
    data=survival_data,
    y='ProtAgeAccel',
    x='ethnicity',
    # palette=colors,
    palette="colorblind",
    inner_kws=inner_kws,
    linewidth=1
)

plt.text(.95, .1, 'UKB', ha='right', va='top', transform=plt.gca().transAxes)

plt.xlabel('')
plt.ylabel('ProtAgeGap (years)')


# plot i: ProtAgeAccel distribution  in both
ax = plt.subplot(gs[2, 2], aspect='auto')  # Span the entire left column
ax.set_title('i', fontweight='bold', loc='left', fontsize=12)

# order alphabetically by rural and then urban
cat_order = [
    'Gansu (Rural)', 
    'Henan (Rural)', 
    'Hunan (Rural)',
    'Sichuan (Rural)',
    'Zhejiang (Rural)',
    'Haikou (Urban)', 
    'Harbin (Urban)', 
    'Liuzhou (Urban)', 
    'Qingdao (Urban)', 
    'Suzhou (Urban)'
]

sns.violinplot(
    data=ckb_normalized,
    y='ProtAgeAccel',
    x='region_code',
    order=cat_order,
    # palette=colors,
    palette="colorblind",
    inner_kws=inner_kws,
    linewidth=1
)

# Rotate the x-axis tick labels at a 45-degree angle
plt.xticks(rotation=45, ha='right')

plt.text(.95, .1, 'CKB', ha='right', va='top', transform=plt.gca().transAxes)

plt.xlabel('')
plt.ylabel('ProtAgeGap (years)')

### final steps
# Adjust the layout
plt.tight_layout()

# Show the plot
# plt.show()

# save
name = f'/home/ivm/Documents/Figure_2.pdf'
plt.savefig(
    name,
    format='pdf',
    dpi=600,
    facecolor='white',
    transparent=False,
    bbox_inches="tight"
)
plt.close()